In [7]:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/get-started/writing-geometries.htm
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/get-raster-properties.htm
# https://gis.stackexchange.com/questions/180757/creating-polygon-feature-class-from-x-y-coordinates-using-arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

# Read input raster
raster = "121.img"

# Get raster extent
myRaster = Raster(raster)   # need to read raster as Raster object
myExtent = myRaster.extent

# Store extent coordinates 
xmax = myExtent.XMax
xmin = myExtent.XMin
ymax = myExtent.YMax
ymin = myExtent.YMin

# Store extent values as list of coordinates 
coordinates = [(xmin, ymin),
               (xmin, ymax),
               (xmax, ymax),
               (xmax, ymin)]

# Get coordinate system
sr = arcpy.Describe(raster).spatialReference

# Create a feature class with a spatial reference of GCS WGS 1984
result = arcpy.management.CreateFeatureclass(
    arcpy.env.scratchGDB, 
    "rasterExtent", 
    "POLYGON", 
    spatial_reference=sr
)


# Create feature class
outPolyExtent= result[0]

# Use Insert cursor to add new geometry to feature class Write feature to new feature class
with arcpy.da.InsertCursor(outPolyExtent, ['SHAPE@']) as cursor:
    cursor.insertRow([coordinates])

In [1]:
#parameters to modify:
# lineInput = "line_Select" jdm: I believe this can just be the selected line
# in our case the CCCL line. Although if we have an area with multiple lines we can modify 
# script do the detect selection fucntionality
lineInput = "CCCLClipProj"
pointOutput = r"G:\My Drive\Projects\FlSeaGrantHurricaneViewsheds\Data\HurricaneIvan\viewshed.gdb\ObserverPoints"
pointSpacing = 10
pointInput = "GeneratePoints"
rasterInput =  "121.img"
observerElevation = "6 Meters"
viewDistance = "1 Miles"
outRaster = r"G:\My Drive\Projects\FlSeaGrantHurricaneViewsheds\Data\HurricaneIvan\viewshed.gdb\Viewshed_121" #FL_G3??

# Generate points along line, 
# Only the part of the viewshed line to that is accounted for w/in the extent of the input raster
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/generate-points-along-lines.htm
arcpy.management.GeneratePointsAlongLines(
    Input_Features= lineInput,
    Output_Feature_Class= pointOutput,
    Point_Placement="PERCENTAGE",
    Percentage=pointSpacing,
    Include_End_Points="NO_END_POINTS",
    Add_Chainage_Fields="NO_CHAINAGE"
)

<Result 'G:\\My Drive\\Projects\\FlSeaGrantHurricaneViewsheds\\Data\\HurricaneIvan\\viewshed.gdb\\ObserverPoints'>

In [75]:
#geodesic viewshed
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_16N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-87.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAD_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', parallelProcessingFactor="100%", scratchWorkspace=r"E:\Projects\viewshed.gdb"):
    out_raster = arcpy.sa.Viewshed2(
        in_raster=rasterInput,
        in_observer_features=pointInput,
        out_agl_raster=None,
        analysis_type="FREQUENCY",
        vertical_error="0 Meters",
        out_observer_region_relationship_table=None,
        refractivity_coefficient=0.13,
        surface_offset="0 Meters",
        observer_elevation=None,
        observer_offset=observerElevation,
        inner_radius=None,
        inner_radius_is_3d="GROUND",
        outer_radius=viewDistance,
        outer_radius_is_3d="GROUND",
        horizontal_start_angle=0,
        horizontal_end_angle=360,
        vertical_upper_angle=180,
        vertical_lower_angle=-180,
        analysis_method="ALL_SIGHTLINES",
        analysis_target_device="CPU_ONLY"
    )
    out_raster.save(outRaster)